# Kalman Filter

Kalman Filter のパッケージとしては pykalman(https://pykalman.github.io) があるが，<br>
Nan や観測データよりも細かいステップでの予測分布の更新など柔軟に対応できるプログラムを自分で作りたいというモチベーション

Kalman Filter について知りたい人は，以下の文献・Qittaを参考にしてほしい．
- 樋口知之編著，上野玄太・中野慎也・中村和幸・吉田亮著，データ同化入門ー次世代のシミュレーション技術ー，朝倉書店，2011．
- Qitta : きみにもわかる，カルマンフィルター，https://qiita.com/deaikei/items/00a2716ecc3e944c139a

## Linear Gauss State Space Model（線形・ガウス状態空間モデル）

カルマンフィルタでは，線形・ガウス状態空間モデルを考える．これは，次の様に表せる．
$$
\begin{align}
&x_{t+1} = F_tx_t + b + N(0,Q_t)\\
&y_t = H_ty_t + d + N(0,R_t)
\end{align}
$$
ここで，$x_t$は時刻$t$における状態変数，$y_t$は時刻$t$における観測変数，$F_t$はシステム行列，$Q_t$はシステムノイズの共分散行列，$H_t$は観測行列，$R_t$は観測ノイズの共分散行列，$b,d$は各モデルのオフセット（切片）を表す．

## Kalman Filter Algorithm

1. 初期状態のフィルタ分布の平均 $x_{0|0}$，共分散行列$V_{0|0}$を与える
1. $t=1$から$T$まで次の予測ステップ・フィルタステップを繰り返す

予測ステップ
$$
\begin{gather}
x_{t|t-1}=F_tx_{t-1|t-1}\\
V_{t|t-1}=F_tV_{t-1|t-1}F_t^T+G_tQ_tG_t
\end{gather}
$$

フィルタステップ
$$
\begin{gather}
K_t = V_{t|t-1}H_t^T (H_tV_{t|t-1}H_t^T+R_t)^{-1}\\
x_{t|t} = x_{t|t-1}+K_t(y_t-H_tx_{t|t-1})\\
V_{t|t} = V_{t|t-1} - K_t H_t V_{t|t-1}
\end{gather}
$$

## RTS Smoothing Algorithm

カルマンフィルタによって，$x_{T|T},V_{T|T}$が得られたら，$t=T-1,\cdots,0$に対して，
$$
\begin{align*}
&A_t=V_{t|t}F_{t+1}^TV_{t+1|t}^{-1}\\
&x_{t|T} = x_{t|t} + A_t(x_{t+1|T} - x_{t+1|t})\\
&V_{t|T} = V_{t|t} + A_t(V_{t+1|T} - V_{t+1|t})A_t^T
\end{align*}
$$
によって固定区間平滑化分布の平均$x_{t|T}$と共分散行列$V_{t|T}$がわかる．

# Let's make program

In [1]:
# install packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [1]:
class Kalman_Filter(object) :
    '''
    <Input Variables>
    observation [time, n_dim_obs] {float} : observation y （観測値）
    initial_mean [time, n_dim_sys] {float} : initial state mean (初期フィルタ分布の平均)
    initial_covariance [n_dim_sys, n_dim_sys] {float} : initial state covariance （初期フィルタ分布の共分散行列）
    transition_matrix [n_dim_sys, n_dim_sys] {float} : transition matrix from x_{t-1} to x_t （システムモデルの変換行列）
    observation_matrix [n_dim_sys, n_dim_obs] {float} : observation matrix （観測行列）
    transition_covariance [n_dim_sys, n_dim_sys] {float} : covariance of system noise （システムノイズの共分散行列）
    observation_covariance [n_dim_obs, n_dim_obs] {float} : covariance of observation noise （観測ノイズの共分散行列）
    transition_offsets [n_dim_sys] {float} : offsets of system transition model （システムモデルの切片 ＝ バイアス = オフセット）
    observation_offsets [n_dim_obs] {float} : offsets of observation model （観測モデルの切片 = バイアス = オフセット）
    n_dim_sys {int} : dimension of system variable （システム変数の次元）
    n_dim_obs {int} : dimension of observation variable （観測変数の次元）
    
    <Variables>
    y [time, n_dim_obs] {float} : observation y （観測値）
    F [n_dim_sys, n_dim_sys] {float} : transition matrix from x_{t-1} to x_t （システムモデルの変換行列）
    Q [n_dim_sys, n_dim_sys] {float} : covariance of system noise （システムノイズの共分散行列）
    b [n_dim_sys] {float} : offsets of system transition model （システムモデルの切片 ＝ バイアス = オフセット）
    H [n_dim_sys, n_dim_obs] {float} : observation matrix （観測行列）
    R [n_dim_obs, n_dim_obs] {float} : covariance of observation noise （観測ノイズの共分散行列）
    d [n_dim_obs] {float} : offsets of observation model （観測モデルの切片 = バイアス = オフセット）
    x_pred [time, n_dim_sys] {float} :  mean of prediction distribution （予測分布の平均）
    V_pred [time, n_dim_sys, n_dim_sys] {float} : covariance of prediction distribution (予測分布の共分散行列)
    x_filt [time, n_dim_sys] {float} : mean of filtering distribution (フィルタ分布の平均)
    V_filt [time, n_dim_sys, n_dim_sys] {float} : covariance of filtering distribution (フィルタ分布の共分散行列)
    x_RTS [time, n_dim_sys] {float} : mean of RTS smoothing distribution (固定区間平滑化分布の平均)
    V_RTS [time, n_dim_sys, n_dim_sys] {float} : covariance of RTS smoothing distribution (固定区間平滑化の共分散行列)
    '''
    
    def __init__(self, observation, initial_mean, initial_covariance, transition_matrix, observation_matrix, 
                 transition_covariance, observation_covariance, transition_offsets = None, 
                 observation_offsets = None, n_dim_sys = None, n_dim_obs = None) :
        if n_dim_obs is None :
            self.y = observation
            self.n_dim_obs = len(self.y[0])
        else :
            self.n_dim_obs = n_dim_obs
            if self.n_dim_obs != len(observation[0]) :
                raise IndexError('You mistake dimension of observation.')
            else :
                self.y = observation
        if n_dim_sys is None :
            self.initial_mean = initial_mean
            self.n_dim_sys = len(self.initial_mean)
        else :
            self.n_dim_sys = n_dim_sys
            if self.n_dim_sys != len(initial_mean) :
                raise IndexError('You mistake dimension of initial mean.')
            else :
                self.initial_mean = initial_mean
        self.initial_covariance = initial_covariance
        self.F = transition_matrix
        self.Q = transition_covariance
        if transition_offsets is None :
            self.b = 0
        else :
            self.b = transition_offsets
        self.H = observation_matrix
        self.R = observation_covariance
        if observation_offsets is None :
            self.d = 0
        else :
            self.d = observation_offsets
        
        # filter function (フィルタ値を計算する関数)
        def Filter(self) :
            '''
            T : length of data y （時系列の長さ）
            K : Kalman gain (カルマンゲイン)
            '''
            T = len(self.y)
            self.x_pred = np.zeros((T + 1, self.n_dim_sys))
            self.V_pred = np.zeros((T + 1, self.n_dim_sys, self.n_dim_sys))
            self.x_filt = np.zeros((T + 1, self.n_dim_sys))
            self.V_filt = np.zeros((T + 1, self.n_dim_sys, self.n_dim_sys))
            
            # initial setting (初期分布)
            self.x_pred[0] = self.initial_mean
            self.V_pred[0] = self.initial_covariance
            self.x_filt[0] = self.initial_mean
            self.V_filt[0] = self.initial_covariance
            
            # GQG^T because G, Q, G is consistent
            GQG = np.dot(self.G, np.dot(self.Q, self.G))
            
            for t in range(T) :
                print("\r filter calculating... t={}".format(t + 1) + "/" + str(T), end="")
                
                # prediction (予測分布)
                self.x_pred[t + 1] = np.dot(self.F, self.x_filt[t]) + self.b
                self.V_pred[t + 1] = np.dot(self.F, np.dot(self.V_filt[t], self.F.T)) + GQG
                
                # filtering (フィルタ分布)
                K = np.dot(V_pred[t + 1], np.dot(self.H.T, np.linalg.inv(np.dot(self.H, np.dot(self.V_pred[t + 1], self.H.T)) + self.R)))
                self.x_filt[t + 1] = np.x_pred[t + 1] + np.dot(K, self.y[t] - np.dot(self.H, self.x_pred[t + 1])) + self.d
                self.V_filt[t + 1] = np.V_pred[t + 1] - np.dot(K, np.dot(self.H, self.V_pred[t + 1]))
        
        # get filtered value (フィルタ値を返す関数，Filter 関数後に値を得たい時)
        def Get_Filtered_Value(self) :
            return self.x_filt[1:]
        
        # RTS smooth function (RTSスムーシングを計算する関数，Filter 関数後に)
        def RTS_Smooth(self) :
            '''
            T : length of data y (時系列の長さ)
            A : fixed interval smoothed gain (固定区間平滑化ゲイン)
            '''
            T = len(self.y)
            self.x_RTS = np.zeros((T + 1, self.n_dim_sys))
            self.V_RTS = np.zeros((T + 1, self.n_dim_sys, self.n_dim_sys))
            
            self.x_RTS[T] = self.x_filt[T]
            self.V_RTS[T] = self.V_filt[T]
            
            # t in [1, T] (tが1~Tの逆順であることに注意)
            for t in range(T, 0, -1) :
                print("\r smooth calculating... t={}".format(T - t + 1) + "/" + str(T), end="")
                
                # fixed interval smoothing (固定区間平滑化分布)
                A = np.dot(self.V_filt[t], np.dot(self.F.T, np.linalg.inv(self.V_pred[t])))
                self.x_RTS[t - 1] = self.x_filt[t] + np.dot(A, self.x_RTS[t] - self.x_pred[t])
                self.V_RTS[t - 1] = self.V_filt[t] + np.dot(A, np.dot(self.V_RTS[t] - self.V_pred[t], A.T))
        
        # get RTS smoothed value (RTS スムーシング値を返す関数，RTS_Smooth 後に)
        def Get_RTS_Smoothed_Value(self) :
            T = len(self.y)
            return self.x_RTS[:T]

これに実際に値を入れるのは今度ということで